In [1]:
import torch
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [3]:
# source from Microsoft Bing Chat
trainX = torch.tensor(train_df.iloc[:, :-1].values, dtype=torch.float64)
trainY = torch.tensor(train_df.iloc[:, -1].values, dtype=torch.float64).view(-1, 1)
testX = torch.tensor(test_df.iloc[:, :-1].values, dtype=torch.float64)
testY = torch.tensor(test_df.iloc[:, -1].values, dtype=torch.float64).view(-1, 1)

In [4]:
#q1

num_bytes = trainX.numel() * 8
num_bytes

83520

In [5]:
#q2

# source from Microsoft Bing Chat
trainX_float16 = trainX.float().half()
trainX_converted = trainX_float16.float().double()
difference = torch.abs(trainX - trainX_converted)
max_difference = torch.max(difference).item()
max_difference

0.0

In [6]:
#q3

cuda_available = torch.cuda.is_available()
cuda_available

False

In [7]:
coef = torch.tensor([
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040], # POS_50_59_CP
        [0.0300], # POS_60_69_CP
        [0.0300],
        [0.0300],
        [0.0300]
], dtype=trainX.dtype)
coef

tensor([[0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0300],
        [0.0300],
        [0.0300],
        [0.0300]], dtype=torch.float64)

In [8]:
#q4

predicted_deaths = torch.matmul(testX[0], coef).item()
predicted_deaths

9.844

In [9]:
#q5

predicted_deaths = torch.matmul(testX, coef)
average_predicted = torch.mean(predicted_deaths).item()
average_predicted

12.073632183908048

In [10]:
#q6

x = torch.tensor(0.0)
y = x**2 - 8*x + 19
float(y)

19.0

In [11]:
#q7

from torch.optim import SGD

# Define the initial value of x
x = torch.tensor([0.0], requires_grad=True)

# Define the optimizer
optimizer = SGD([x], lr=0.016)

# Optimization loop
for i in range(35000):
    y = x**2 - 8*x + 19  # Calculate y
    loss = (y - 3)**2  # source from Microsoft Bing Chat  # goal is to find an x s.t. y is close to 3
    loss.backward()  # Compute the gradients
    optimizer.step()  # Update x
    optimizer.zero_grad()

x.item()

4.015500545501709

In [12]:
#q8

# Initialize coef with zeros
coef = torch.zeros((trainX.size(1), 1), dtype=trainX.dtype)

# Compute predictions
predictions = trainX @ coef

# Compute MSE
mse = torch.mean((predictions - trainY)**2).item()
mse

197.8007662835249

In [13]:
#q9

from torch.utils.data import TensorDataset, DataLoader

# Seed 544
torch.manual_seed(544)

# Setup a training dataset and data loader
ds = TensorDataset(trainX, trainY)
dl = DataLoader(ds, batch_size=50, shuffle=True)

# Initialize coef with zeros
# trainX.size(1) = 10
coef = torch.zeros((10,1), dtype=torch.float64, requires_grad=True)

# Define the optimizer and loss function
optimizer = SGD([coef], lr=0.000002)
loss_fn = torch.nn.MSELoss()

# Training loop
for epoch in range(500):
    for X, y in dl:   # source from Microsoft Bing Chat # iterates over the dl 
        predictions = X @ coef  
        loss = loss_fn(predictions, y) 
        loss.backward()  
        optimizer.step()  
        optimizer.zero_grad()

# Compute MSE over the training data using the optimized coefficients
predictions = trainX @ coef.detach()
mse = loss_fn(predictions, trainY).item()
mse

26.8113940147193

In [14]:
#q10

# Compute MSE over the test data using the optimized coefficients
test_predictions = testX @ coef.detach()
test_mse = loss_fn(test_predictions, testY).item()
test_mse

29.05854692548551